In [5]:
import numpy as np
import matplotlib.pyplot as plt

In [191]:
class RestrictedBoltzmannMachine:
    def __init__(self, n_visible: int, n_hidden: int):
        '''
        Initialize 2 Layer Restricted Boltzmann Machine with a specified number of hidden nodes and visible nodes.

        Arguments:
          n_visible (int) - Number of nodes in the visible layer
          n_hidden  (int) - Number of nodes in the hidden layer
        '''
        self.n_visible = n_visible
        self.n_hidden  = n_hidden
        self.W = np.random.rand(n_visible, n_hidden) # Set to this for testing purposes - np.array([[2, -1, 1], [-2, 0, -1]])
        
    def sigmoid(self, x):
        '''
        Activation function for output of nodes in either layer.
        '''
        return 1 / (1 + np.exp(-x))
    
    def reconstruct_hidden_layer(self, visible_input):
        # Calculate hidden probabilities
        activation = np.vectorize(self.sigmoid)
        hidden_probas = activation(np.inner(visible_input, self.W.T))

        # Sample hidden layer output from calculated probabilties
        random_vars = np.random.random(size=self.n_hidden) # Set to this for testing purposes - np.array([0.87, 0.14, 0.64]) 
        return (random_vars < hidden_probas).astype(int)

    def reconstruct_visible_layer(self, hidden_input):
        # Calculate visible probabilities
        activation = np.vectorize(self.sigmoid)
        visible_probas = activation(np.inner(hidden_input, self.W))

        # Sample visible layer output from calculated probabilties
        random_vars = np.random.random(size=self.n_visible) # Set to this for testing purposes - np.array([0.25, 0.72])
        return (random_vars < visible_probas).astype(int)
    
    def train(self, patterns, eta=0.01):
        for v in patterns:
            # Take training sample, and comput probabilties of hidden units and the
            # sample a hidden activation vector from this probability distribution
            h = self.reconstruct_hidden_layer(v)

            # Compute the outer product of v and h and call this the positive gradient
            pos_grad = np.outer(v, h)

            # From h, sample a reconstruction v' of the visible units
            v_prime = self.reconstruct_visible_layer(h)
            
            # Then resample activations h' from this
            h_prime = self.reconstruct_hidden_layer(v_prime)

            # Compute the outer product of v' and h' and call this the negative gradient
            neg_grad = np.outer(v_prime, h_prime)

            # The update to the weight matrix W, will be the positive gradient minus the negative gradient, times some learning rate
            W_delta = eta*(pos_grad - neg_grad)

            self.W = np.subtract(self.W, W_delta)
            
    def recall(self, pattern, iters=100):
        v = pattern
        for _ in range(iters):
            h = self.reconstruct_hidden_layer(v)
            v = self.reconstruct_visible_layer(h)
        
        return v

In [192]:
rbm = RestrictedBoltzmannMachine(2, 3)
rbm.train([[1,1]])

In [118]:
t = np.array([
        [-1, -1, -1, -1, -1, -1, -1, -1,  1,  1],
        [-1, -1, -1, -1, -1, -1, -1,  1,  1,  1],
        [-1, -1, -1, -1, -1, -1,  1,  1, -1,  1],
        [-1, -1, -1, -1, -1,  1,  1,  1, -1,  1],
        [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [-1, -1, -1, -1, -1, -1, -1, -1,  1,  1],
        [-1, -1, -1, -1, -1, -1, -1, -1,  1,  1],
        [-1, -1, -1, -1, -1, -1, -1, -1,  1,  1],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
    ])